In [1]:
import time
import typing as t
from openpyxl import load_workbook
from datetime import datetime
from pydantic import BaseModel

from app.core.config import settings
from app.db.session import SessionLocal
from app import crud, models, schemas
db = SessionLocal()

/usr/local/lib/python3.7/site-packages/jose/backends/cryptography_backend.py:18: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes, int_to_bytes


In [2]:
def get_file(file="reklaliste.xlsx") -> load_workbook:
    wb_file = load_workbook(str(settings.SERVER_BASE_DIR) + '/' + file)
    i = 0
    for sheet in wb_file.sheetnames:
        i += 1
        if sheet.lower() == "liste wertreklas":
            break
        wb_file.active = i
    return wb_file.active

def create_claim(*, data: t.List) -> t.Union[schemas.claim.ClaimBase, None]:
    claim_in = schemas.claim.ClaimBase()
    if isinstance(data[1], float):
        contract_nr = str(data[1]).split(".0")[0]
        claim_in.contract_nr = int(contract_nr)

    if isinstance(data[2], float) or isinstance(data[2], int):
        claim_in.discharge = float(data[2])

    if isinstance(data[3], float) or isinstance(data[3], int):
        claim_in.bill = float(data[3])

    if isinstance(data[7], datetime):
        claim_in.created_at = datetime.date(data[7])

    if isinstance(data[11], str):
        kind = data[11].strip().upper()
        try:
            claim_in.kind = models.ClaimKind(kind).name
        except ValueError:
            claim_in.kind = None    

    if claim_in.contract_nr is not None and claim_in.bill is not None and claim_in.created_at is not None:
        return claim_in
    return None

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
claims_file = get_file()
def fetch_claim_data(row: t.List) -> t.List:
    data = []
    for cell in row:
        data.append(cell)
    claim_obj = create_claim(data=data)
    return claim_obj

/usr/local/lib/python3.7/site-packages/openpyxl/styles/stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
def iter_claim_rows(file: load_workbook) -> t.Callable:
    i = 0
    for row in file.iter_rows(values_only=True, max_col=15):
        i += 1
        if i % 1000 == 0:
            print("...calma 5 seconds!")
            time.sleep(5)
        echo = fetch_claim_data(row)
        if echo is not None:
            yield echo

In [10]:
rows = iter_claim_rows(file=claims_file)

In [11]:
    for row in rows:
        ticket_exist = crud.ticket.get_by_kwargs(db, contract_nr=row.contract_nr,
                                                 kind=models.TicketKind.COMMISSION).first()
        if ticket_exist:
            # UPDATE CLAIM DATA
            row.store_internal_id = ticket_exist.store_internal_id
            row.owner_id = ticket_exist.owner_id
            row.ticket_id = ticket_exist.ticket_id

            print("...ticket ->", ticket_exist.ticket_id)
            claim_exist = crud.claim.get_by_kwargs(db, contract_nr=row.contract_nr, created_at=row.created_at, bill=row.bill).first()
            if not claim_exist:
                print("...claim not exist!")
                claim = crud.claim.create(db, obj_in=row)
                if claim:
                    print(f"...created new claim {claim.id}")
            else:
                print("...claim exist!")
                claim = crud.claim.update(db, db_obj=claim_exist, obj_in=row)
                if claim:
                    print(f"...updated claim {claim.id}")

InvalidRequestError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (psycopg2.errors.InvalidTextRepresentation) invalid input value for enum claimkind: "CUSTOMER"
LINE 1: ...VALUES ('2020-01-28', NULL, 1211269, 76.71, NULL, 'CUSTOMER'...
                                                             ^

[SQL: INSERT INTO claim (created_at, updated_at, contract_nr, bill, discharge, kind, ticket_id, owner_id, store_internal_id) VALUES (%(created_at)s, %(updated_at)s, %(contract_nr)s, %(bill)s, %(discharge)s, %(kind)s, %(ticket_id)s, %(owner_id)s, %(store_internal_id)s) RETURNING claim.id]
[parameters: {'created_at': '2020-01-28', 'updated_at': None, 'contract_nr': 1211269, 'bill': 76.71, 'discharge': None, 'kind': 'CUSTOMER', 'ticket_id': 37865, 'owner_id': 1, 'store_internal_id': 188}]
(Background on this error at: http://sqlalche.me/e/13/9h9h) (B

In [12]:
db.close()

In [13]:
db = SessionLocal()

In [14]:
    for row in rows:
        ticket_exist = crud.ticket.get_by_kwargs(db, contract_nr=row.contract_nr,
                                                 kind=models.TicketKind.COMMISSION).first()
        if ticket_exist:
            # UPDATE CLAIM DATA
            row.store_internal_id = ticket_exist.store_internal_id
            row.owner_id = ticket_exist.owner_id
            row.ticket_id = ticket_exist.ticket_id

            print("...ticket ->", ticket_exist.ticket_id)
            claim_exist = crud.claim.get_by_kwargs(db, contract_nr=row.contract_nr, created_at=row.created_at, bill=row.bill).first()
            if not claim_exist:
                print("...claim not exist!")
                claim = crud.claim.create(db, obj_in=row)
                if claim:
                    print(f"...created new claim {claim.id}")
            else:
                print("...claim exist!")
                claim = crud.claim.update(db, db_obj=claim_exist, obj_in=row)
                if claim:
                    print(f"...updated claim {claim.id}")

...ticket -> 37725
...claim exist!
...updated claim 1
...ticket -> 36766
...claim exist!
...updated claim 2
...ticket -> 36509
...claim exist!
...updated claim 3
...ticket -> 36507
...claim exist!
...updated claim 4
...calma 5 seconds!
...ticket -> 37950
...claim exist!
...updated claim 5
...ticket -> 37103
...claim exist!
...updated claim 6
...ticket -> 37108
...claim exist!
...updated claim 7
...ticket -> 37108
...claim exist!
...updated claim 8
...ticket -> 37629
...claim exist!
...updated claim 9
...ticket -> 37865
...claim not exist!
...created new claim 10
...ticket -> 39117
...claim not exist!
...created new claim 11
...ticket -> 37952
...claim not exist!
...created new claim 12
...ticket -> 37971
...claim not exist!
...created new claim 13
...ticket -> 37955
...claim not exist!
...created new claim 14
...ticket -> 36487
...claim not exist!
...created new claim 15
...ticket -> 38526
...claim not exist!
...created new claim 16
...ticket -> 37974
...claim not exist!
...created new

In [15]:
from sqlalchemy.sql import func

In [16]:
models.Claim.query(db)

AttributeError: type object 'Claim' has no attribute 'query'

In [17]:
models.claim.Claim.query()

AttributeError: type object 'Claim' has no attribute 'query'

In [18]:
db.query()

In [25]:
x = db.query(func.avg(models.Claim.bill).label('average')).first()

In [26]:
x

(93.68792328042325)

In [27]:
y = db.query(func.sum(models.Claim.bill).label('sum')).first()

In [28]:
y

(70828.06999999998)

In [33]:
ba = db.query(func.sum(models.Claim.discharge).label('ba')).filter

In [41]:
claims_ba = crud.claim.get_query(db).with_entities(func.sum(models.Claim.bill).label('bill')).first()

In [71]:
claims_ba

(70828.06999999998)

In [72]:
without_discharge = crud.claim.get_query(db).filter(models.Claim.discharge.is_(None)).all()

InternalError: (psycopg2.errors.InFailedSqlTransaction) current transaction is aborted, commands ignored until end of transaction block

[SQL: SELECT claim.created_at AS claim_created_at, claim.updated_at AS claim_updated_at, claim.id AS claim_id, claim.contract_nr AS claim_contract_nr, claim.bill AS claim_bill, claim.discharge AS claim_discharge, claim.kind AS claim_kind, claim.ticket_id AS claim_ticket_id, claim.owner_id AS claim_owner_id, claim.store_internal_id AS claim_store_internal_id 
FROM claim 
WHERE claim.discharge IS NULL]
(Background on this error at: http://sqlalche.me/e/13/2j85)